In [ ]:
import pandas as pd

In [ ]:
intranet_df = pd.read_csv("data/intranet_data.tsv", encoding="utf-8", sep="\t")

In [ ]:
keyword_df = intranet_df.copy()
keywords = ["Maske", "Mundnasenschutz", "FFP2", "Beschränkung", "Verbot", "Besuch", "Corona", "Covid", "Infektion", "Maßnahme", "Pflicht", "Gebot", "Hygiene", "Abstand"]
for keyword in keywords:
    keyword_df[keyword] = keyword_df.message.str.lower().str.contains(keyword.lower())

# n_matches refers to the number of matched keywords per message
keyword_df["n_matches"] = keyword_df[keywords].sum(axis=1)

In [ ]:
# calculate cumulative sum of counts
keyword_count = keyword_df.groupby("n_matches").size().reset_index(name="count")
keyword_count = keyword_count.sort_values("n_matches", ascending=False)
keyword_count["cumulative_sum"] = keyword_count["count"].cumsum()
keyword_count

In [ ]:
import matplotlib.pyplot as plt
plt.plot(
    "n_matches",
    "cumulative_sum",
    data=keyword_count.loc[keyword_count.n_matches > 0],
)
plt.xlabel("≥ n distinct keywords in a message")
plt.ylabel("Number of messages")
plt.title("Number of messages vs. Distinct keywords per message")

Setting the threshold of required keywords in a message to ≥ 4 seems reasonable.

## Calculating the ratio of keywords in the filtered messages
Which of the keywords occur more often in the messages with more than 2 distinct keywords?

In [ ]:
keyword_frequency = pd.DataFrame({"keyword": keywords, "frequency": keyword_df[keywords].sum()}).reset_index(drop=True)
keyword_frequency = keyword_frequency.sort_values(by="frequency", ascending=False)
keyword_frequency

In [ ]:
keyword_frequency_4_matches = pd.DataFrame({"keyword": keywords, "frequency": keyword_df.loc[keyword_df.n_matches > 3][keywords].sum(axis=0)}).reset_index(drop=True)
keyword_frequency_4_matches = keyword_frequency_4_matches.sort_values(by="frequency", ascending=False)
keyword_frequency_4_matches

In [ ]:
keyword_factor = keyword_frequency_4_matches.copy()
keyword_factor = keyword_factor.rename(columns={"frequency": "frequency_4_matches"})
keyword_factor = pd.merge(keyword_factor, keyword_frequency, on="keyword")
keyword_factor.frequency_4_matches = keyword_factor.frequency_4_matches.astype(float)
keyword_factor.frequency = keyword_factor.frequency.astype(float)

# calculate the ratio of the frequency of keywords in messages with 3 or more keywords vs all messages
keyword_factor["ratio"] = round(keyword_factor.frequency_4_matches / keyword_factor.frequency, 2)
keyword_factor.sort_values("ratio", ascending=False)

# Anonymize messages
Before we process the messages using OpenAI's GPT API, we anonymize the messages by deleting all names.

In [ ]:
keyword_df_4_or_more_matches = keyword_df.loc[keyword_df.n_matches > 3].reset_index(drop=True)

In [ ]:
import spacy
nlp = spacy.load("de_core_news_lg")
#import de_dep_news_trf
#nlp = de_dep_news_trf.load()

keywords_lower = [x.lower() for x in keywords]

def remove_names(text):
    # Process the text through the spaCy NLP pipeline
    doc = nlp(text)
    # Iterate over the detected entities
    for ent in doc.ents:
        # Check if the entity is a person's name
        if ent.label_ == "PER":
            if any(keyword in ent.text.lower().strip() for keyword in keywords_lower):
                print(f"NOT replaced: {ent.text}")
                continue
            # Replace the person's name with an empty string
            text = text.replace(ent.text, 'NAME ENTITY')
            print(f"Succesfully replaced: {ent.text}")
    return text

In [ ]:
keyword_df_4_or_more_matches["message_anonymized"] = keyword_df_4_or_more_matches.message.map(remove_names)

In [ ]:
keyword_df_4_or_more_matches

## OpenAI for measure classification
The following code will be used to classify the messages content into "relaxation", "tightening" or "unclear".

In [ ]:
from openai import OpenAI
from os import getenv
import json

client = OpenAI(api_key=getenv("OPENAI_API_KEY"))
# Helper function to send messages to OpenAI API (ChatGPT model)
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        messages=messages,
        model="gpt-3.5-turbo",
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content.replace('```', '')

In [ ]:
prompt_few_shot = (
    'I will give you a text that stems from the intranet of a hospital. '
    'The text may contain information on changes in certain measures regarding the antiinfectious management to prevent the spread of respiratory infections in the hospital. '
    'Your task is to identify whether the text states that the measures are being significantly tightened or relaxed, or if there are no changes described and the existing measures are being reminded of. '
    'Please return the answer as a JSON object in the following format: {{"classification": <label>}} without any explanations. '
    'The valid options for the label are: "tightened", "relaxed", "unclear".\n\n'
    'Here are some examples:\n\n'
    'Input: ```… Die Geschäftsführung und Klinikumsleitung haben folgende Anpassungen der Corona-Verfahrensanweisungen für Mitarbeitenden und Patienten für das Klinikum EvB, Campus Potsdam vorgenommen. Damit fallen auch die letzten Testregelungen für die Patient*innen. Die wichtigsten Änderungen, die ab dem 1. Mai gelten, finden Sie hier im Überblick: • Testungen von Patient*innen entfallen ab dem 1.5.2023:- keine Routine-Testungen mehr bei der Aufnahme- keine Testung mehr bei Notfallpatienten - kein Verlaufsscreening mehr • Testungen erfolgen jedoch weiterhin bei Verdacht nach klinischer Maßgabe.``` Output: ```{{"classification": "relaxed"}}```\n\n'
    'Input: ```Aufgrund der aktuellen Situation in der Ukraine ist die Lage im Cyberraum angespannt. Die Ukraine ist seit mehreren Jahren Opfer regelmäßiger Cyberangriffe und die Aktivitäten haben sich seit Anfang des Kriegsgeschehens im Februar intensiviert, da die militärischen Operationen weiterhin durch Maßnahmen im Cyberraum begleitet werden. Obwohl noch kein besonderer Anstieg von Cyberangriffe in Deutschland zu verzeichnen ist, erwarten IT-Sicherheitsexperten in Europa, dass vor allem Institutionen, die die Ukraine unterstützen, Opfer von Cyberangriffen werden. Das gilt insbesondere für Unternehmen der kritischen Infrastruktur wie Energie, Wasser, Telekommunikation, Banken, Krankenhäuser, Logistik, etc. Daher implementiert auch die  neue IT-Sicherheitsmaßnahmen, um das Risiko direkter Angriffe aus Russland und Belarus zu minimieren. Viele der Maßnahmen werden Sie im Rahmen Ihrer täglichen Arbeit nicht bemerken.Eine Maßnahme ist jedoch das Sperren aller Webseiten, die in Russland und Belarus gehostet sind. Bitte bedenken Sie dies beim Aufruf von Webseiten. Die erhöhte Bedrohungslage für Deutschland im Zusammenhang mit dem Krieg gegen die Ukraine wird für die nächsten Monaten bestehen bleiben. Deswegen ist es besonders wichtig, dass Sie:- die zwei Pflichtschulungen zum Thema IT-Sicherheit und Phishing-Mails im E-Learning-System jetzt absolvieren, wenn Sie es noch nicht gemacht haben- weiterhin Phishing-Mails melden- vermeiden, nicht vertrauenswürdige Webseiten zu besuchen- dem Helpdesk Störungen melden Mit freundlichen Grüßen  (ISB).``` Output: ```{{"classification": "unclear"}}```\n\n'
    'Input: ```• Aufhebung der Maskenpflicht für Besuchende im Klinikum • Aufhebung der Maskenpflicht für Patient*innen der ambulanten Praxen (Poliklinik)• KEvB hält an Tests von stationären Patient*innen während des Klinikaufenthaltes fest Zum Karfreitag, 7. April 2023 endet das bundesweit geltende Infektionsschutzgesetz § 28 „Besondere Schutzmaßnahmen zur Verhinderung der Verbreitung der Coronavirus-Krankheit-2019“ und damit auch die Maskenpflicht in Krankenhäusern und Arztpraxen. Somit gelten im Klinikum Ernst von Bergmann sowie dem Klinikum Westbrandenburg, Standort Potsdam, ab Karfreitag, 7. April keine coronaspezifischen Besuchsregelungen mehr: Besuchsregelung stationäre Patient*innen- Besucher dürfen ohne Maskenpflicht und ohne Testpflicht das Haus betreten.- Besuche von isolationspflichtigen Patient*innen (dazu zählt auch das Sars-CoV-2-Virus), sowie die Begleitung von Patient*innen in der Notaufnahme, sind auch weiterhin nur in besonderen Situationen und Lebenslagen nach Absprache möglich.- Ein Besuchsverbot gilt für Stationen mit aktivem Ausbruchsgeschehen.- Sonderregelungen einzelner Kliniken und Fachbereichen sind individuell über die Homepage der Klinik oder der Station zu erfragen. Ambulante Patient*innen (Poliklinik und ambulante Arztpraxen)- Patient*innen dürfen die Arztpraxen ohne Maskenpflicht und ohne Testpflicht betreten Stationäre Patient*innen- Stationäre Patient*innen werden durch das KEvB bei Aufnahme auf Sars-CoV-2 getestet. Testungen im Verlauf erfolgen bei klinischer Symptomatik und Langliegern. Diese Maßnahme wird regelmäßig klinikintern neu bewertet.- Personen, die auf der Geburtsstation als Begleitung im Familienzimmer mit aufgenommen werden, werden ebenfalls bei Aufnahme auf Sars-CoV-2 getestet. Alle Informationen finden Sie ebenfalls über: www.evb-gesundheit.de/klinikumevb/besuch/corona-besuchsregeln``` Output: ```{{"classification": "relaxed"}}```\n\n'
    'Input: ```Der Krisenstab hat in seiner Sitzung am Freitag Anpassungen im Pandemie-Management im Hinblick auf Omikron beschlossen. Hier finden Sie einen Überblick über die ab Donnerstag, 13.01.2022 geltenden Regelungen und Hinweise zum Hygiene- und Abstrichregelungen sowie Quarantäne und Isolationszeiten. 1. Mitarbeitenden-Schutz – VA Mitarbeitermanagement (url:consense://ConSense_Produktiv/D25135600): - Die Testfrequenz für immunisierte Mitarbeitende ist auf 3x/wöchentliche Antigentests erhöht worden. Weiterhin besteht die Möglichkeit, einen verpflichtenden Antigentest durch einen PCR-Test in der zentralen Abstrichstelle (Haus V) zu ersetzen. - FFP2-Maske ist durchgängig am Arbeitsplatz (Arztbüros, Pflegestützpunkte, Büros, Besprechungen, Dokumentation) zu tragen, insbesondere wenn mehre Personen im Raum sind bzw. in dem Raum zusammenkommen. - Führen Sie möglichst alle Meetings, Jour Fixe oder interne Weiterbildungen digital durch. - Sollten Sie dazu noch Webex-Zugangsdaten benötigen, stellen Sie einen Antrag über das Antragsportal (http://prdweb01.kevb.lan/prod). - Führen Sie Präsenzmeetings nur als Ausnahme mit möglichst geringer Teilnehmerzahl durch. - Durchgängiges Tragen der FFP2-Maske bei allen Präsenzveranstaltungen im Klinikum - FFP2-Maske ist von allen Teilnehmenden und Dozenten/Trainern durchgängig zu tragen. - Auch am Sitzplatz oder bei Veranstaltungen nach dem 2G-Plus-Modell sind die FFP2-Masken durchgängig zu tragen - Anpassung der Isolations- und Quarantänebestimmungen: - Die neusten Entscheidungen von Bund und Ländern zu den neuen Quarantäneregeln müssen durch den Bundestag und Bundesrat noch beschlossen und dann durch das Land Brandenburg in für uns geltende Regelungen übertragen werden. Im Vorfeld wurden bereits die Eckpunkte der neuen Regelungen mit dem Gesundheitsamt Potsdam abgestimmt und in die VA eingebracht:``` Output: ```{{"classification": "tightened"}}```\n\n'
    'Input: ```{}``` Output: '
)

In [ ]:
classification_df = keyword_df_4_or_more_matches.copy()
#pd.DataFrame(columns=["message_anonymized", "classification"])
for i, message in enumerate(keyword_df_4_or_more_matches["message_anonymized"]):
    completion = get_completion(prompt_few_shot.format(message))
    jsoned = json.loads(completion)
    classification_df.loc[i, "classification"] = jsoned["classification"]
    classification_df.loc[i, "gpt35_response"] = completion

When trying out GPT-4 we got some results where the classification was "none" due to some internal error of the GPT. After manually reviewing cases where the classification was "none", we came to the conclusion to set these cases to "unclear" as they were all messages containing reminders of already existing rules, without any changes in the measures themselves.
However, in the end we used GPT3.5-Turbo where we did not encounter these errors.

In [ ]:
classification_df.loc[classification_df.classification == "none", "classification"] = "unclear" 
classification_df.groupby("classification").size()

In [ ]:
classification_df.to_csv("data/corona_measures_classification_gpt35.tsv", encoding="utf-8", sep="\t", index=False)